In [ ]:
%%writefile merge.cu
#include <iostream>
#include <vector>
#include <chrono>
#include <cuda_runtime.h>

using namespace std;

// CUDA kernel for Merge Sort
__device__ void merge(int* arr, int l, int m, int r) {
    int i, j, k;
    int n1 = m - l + 1;
    int n2 = r - m;

    // Create temporary arrays
    int* L = new int[n1];
    int* R = new int[n2];

    // Copy data to temporary arrays L[] and R[]
    for (i = 0; i < n1; i++)
        L[i] = arr[l + i];
    for (j = 0; j < n2; j++)
        R[j] = arr[m + 1 + j];

    // Merge the temporary arrays back into arr[l..r]
    i = 0; // Initial index of first subarray
    j = 0; // Initial index of second subarray
    k = l; // Initial index of merged subarray
    while (i < n1 && j < n2) {
        if (L[i] <= R[j]) {
            arr[k] = L[i];
            i++;
        }
        else {
            arr[k] = R[j];
            j++;
        }
        k++;
    }

    // Copy the remaining elements of L[], if any
    while (i < n1) {
        arr[k] = L[i];
        i++;
        k++;
    }

    // Copy the remaining elements of R[], if any
    while (j < n2) {
        arr[k] = R[j];
        j++;
        k++;
    }

    delete[] L;
    delete[] R;
}

__global__ void merge_sort(int* arr, int size) {
    // Each thread sorts a different segment of the array
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    int l, r;

    // Divide the array into segments for each thread
    int segment_size = (size + gridDim.x - 1) / gridDim.x;
    l = tid * segment_size;
    r = min((tid + 1) * segment_size - 1, size - 1);

    // Perform iterative merge sort on each segment
    for (int current_size = 1; current_size <= r - l; current_size *= 2) {
        for (int left_start = l; left_start < r; left_start += 2 * current_size) {
            int mid = min(left_start + current_size - 1, r);
            int right_end = min(left_start + 2 * current_size - 1, r);
            merge(arr, left_start, mid, right_end);
        }
    }
}

// Host function to launch merge_sort kernel
void launch_merge_sort(int* arr, int size) {
    int block_size = 256;
    int grid_size = (size + block_size - 1) / block_size;
    merge_sort<<<grid_size, block_size>>>(arr, size);
    cudaDeviceSynchronize();
}

// Merge function for sequential merge sort
void sequential_merge(int* arr, int l, int m, int r) {
    int i, j, k;
    int n1 = m - l + 1;
    int n2 = r - m;

    // Create temporary arrays
    int* L = new int[n1];
    int* R = new int[n2];

    // Copy data to temporary arrays L[] and R[]
    for (i = 0; i < n1; i++)
        L[i] = arr[l + i];
    for (j = 0; j < n2; j++)
        R[j] = arr[m + 1 + j];

    // Merge the temporary arrays back into arr[l..r]
    i = 0; // Initial index of first subarray
    j = 0; // Initial index of second subarray
    k = l; // Initial index of merged subarray
    while (i < n1 && j < n2) {
        if (L[i] <= R[j]) {
            arr[k] = L[i];
            i++;
        }
        else {
            arr[k] = R[j];
            j++;
        }
        k++;
    }

    // Copy the remaining elements of L[], if any
    while (i < n1) {
        arr[k] = L[i];
        i++;
        k++;
    }

    // Copy the remaining elements of R[], if any
    while (j < n2) {
        arr[k] = R[j];
        j++;
        k++;
    }

    delete[] L;
    delete[] R;
}

// Sequential merge sort function
void sequential_merge_sort(int* arr, int l, int r) {
    if (l < r) {
        int m = l + (r - l) / 2;

        // Sort first and second halves recursively
        sequential_merge_sort(arr, l, m);
        sequential_merge_sort(arr, m + 1, r);

        // Merge the sorted halves
        sequential_merge(arr, l, m, r);
    }
}

int main() {
    // Test data
    int arr[] = {64, 34, 25, 12, 22, 11, 90};
    int size = sizeof(arr) / sizeof(arr[0]);

    // Merge Sort on GPU
    int* d_arr;
    cudaMalloc(&d_arr, size * sizeof(int));
    cudaMemcpy(d_arr, arr, size * sizeof(int), cudaMemcpyHostToDevice);

    auto start_gpu = chrono::high_resolution_clock::now();
    launch_merge_sort(d_arr, size);
    cudaDeviceSynchronize();
    auto end_gpu = chrono::high_resolution_clock::now();
    chrono::duration<double> duration_gpu = end_gpu - start_gpu;
    cout << "Parallel Merge Sort took " << duration_gpu.count() << " seconds\n";

    cudaMemcpy(arr, d_arr, size * sizeof(int), cudaMemcpyDeviceToHost);

    // Display sorted array from GPU
    cout << "Sorted Array (Parallel): ";
    for (int i = 0; i < size; i++) {
        cout << arr[i] << " ";
    }
    cout << endl;

    cudaFree(d_arr);

    // Sequential Merge Sort
    auto start_cpu = chrono::high_resolution_clock::now();
    sequential_merge_sort(arr, 0, size - 1);
    auto end_cpu = chrono::high_resolution_clock::now();
    chrono::duration<double> duration_cpu = end_cpu - start_cpu;
    cout << "Sequential Merge Sort took " << duration_cpu.count() << " seconds\n";

    // Display sorted array from CPU
    cout << "Sorted Array (Sequential): ";
    for (int i = 0; i < size; i++) {
        cout << arr[i] << " ";
    }
    cout << endl;

    return 0;
}


Overwriting merge.cu


In [ ]:
!nvcc merge.cu -o merge

In [ ]:
!./merge

Parallel Merge Sort took 0.00142819 seconds
Sorted Array (Parallel): 11 12 22 25 34 64 90 
Sequential Merge Sort took 8.393e-06 seconds
Sorted Array (Sequential): 11 12 22 25 34 64 90 


In [ ]:
#include <iostream>
#include <vector>
#include <chrono>
#include <cuda_runtime.h>